In [3]:
import os
base_dir = '/home/olivia/Documents/tractTracker'
os.chdir(base_dir)
#comment after installed once
#!git clone https://github.com/MIC-DKFZ/nnUNet.git 
#!git clone https://github.com/prateekgupta891/nnUNet.git #my forked version
#!git clone https://github.com/NVIDIA/apex

In [4]:
respository_dir = os.path.join(base_dir,'nnUNet')
os.chdir(respository_dir)

#!pip install -e .
#(optional installation)
#!pip install --upgrade git+https://github.com/nanohanno/hiddenlayer.git@bugfix/get_trace_graph#egg=hiddenlayer

os.chdir(base_dir)

In [5]:

#libraries
import shutil
from collections import OrderedDict
import json
import numpy as np

#visualization of the dataset
import matplotlib.pyplot as plt
import nibabel as nib

In [6]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f'{folder_path} exists.')
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

In [7]:

task_name = 'Task101_IC' #change here for different task name
nnunet_dir = "nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data"
task_folder_name = os.path.join(nnunet_dir,task_name)
train_image_dir = os.path.join(task_folder_name,'imagesTr')
train_label_dir = os.path.join(task_folder_name,'labelsTr')
test_dir = os.path.join(task_folder_name,'imagesTs')
main_dir = os.path.join(base_dir,'nnUNet/nnunet')

In [7]:

make_if_dont_exist(task_folder_name,overwrite = False)
make_if_dont_exist(train_image_dir)
make_if_dont_exist(train_label_dir)
make_if_dont_exist(test_dir,overwrite= False)
make_if_dont_exist(os.path.join(main_dir,'nnunet_trained_models'))

nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task101_IC created!
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task101_IC/imagesTr created!
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task101_IC/labelsTr created!
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task101_IC/imagesTs created!
/home/olivia/Documents/tractTracker/nnUNet/nnunet/nnunet_trained_models created!


In [8]:

os.environ['nnUNet_raw_data_base'] = os.path.join(main_dir,'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = os.path.join(main_dir,'preprocessed')
os.environ['RESULTS_FOLDER'] = os.path.join(main_dir,'nnUNet_trained_models')

In [9]:
def copy_and_rename(old_location,old_file_name,new_location,new_filename,delete_original = False):

    shutil.copy(os.path.join(old_location,old_file_name),new_location)
    os.rename(os.path.join(new_location,old_file_name),os.path.join(new_location,new_filename))
    if delete_original:
        os.remove(os.path.join(old_location,old_file_name))

In [12]:

os.chdir(task_folder_name)
if os.path.isfile('training.zip'):
    print(f'Training file for exists')
else:
    print('Training file for SCGM Challenge is not present in the directory')

if os.path.isfile('testing.zip'):
    print('Testing file for SCGM Challenge exists')
else:
    print('Testing file for SCGM Challenge is not present in the directory')
os.chdir(base_dir)

Training file for exists
Testing file for SCGM Challenge exists


In [31]:
#unzipping in nnUNet_raw folder the training data
os.chdir(task_folder_name)
!unzip training.zip
os.chdir(base_dir)

Archive:  training.zip
  inflating: __MACOSX/._2D scans masks training  
  inflating: 2D scans masks training/2025_14_0000.nii.gz  
  inflating: 2D scans masks training/2066_14_mask.nii.gz  
  inflating: 2D scans masks training/2213_16_mask.nii.gz  
  inflating: 2D scans masks training/2389_27_mask.nii.gz  
  inflating: 2D scans masks training/2035_15_mask.nii.gz  
  inflating: 2D scans masks training/2257_26_mask.nii.gz  
  inflating: 2D scans masks training/2338_14_mask.nii.gz  
  inflating: 2D scans masks training/2086_17_mask.nii.gz  
  inflating: 2D scans masks training/2227_28_0000.nii.gz  
  inflating: 2D scans masks training/2031_34_mask.nii.gz  
  inflating: 2D scans masks training/2170_14_mask.nii.gz  
  inflating: 2D scans masks training/2107_3_0000.nii.gz  
  inflating: 2D scans masks training/2175_15_0000.nii.gz  
  inflating: 2D scans masks training/2136_15_mask.nii.gz  
  inflating: 2D scans masks training/2146_16_0001.nii.gz  
  inflating: 2D scans masks training/2306_1

In [32]:
#putting training images into folder

mask_count = 1 #change if more mask is available
zip_path = os.path.join(task_folder_name,"2D scans masks training")
for file in os.listdir(zip_path):
    
    if file.endswith('.nii.gz'):
        if file.find('mask')!=-1:
            #putting mask
            shutil.move(os.path.join(zip_path,file),train_label_dir)
        else:
            #making 4 copies
            if file.find('0000.nii')!=-1:
                for mask in range(1,mask_count+1):
                    #new_filename = file[:file.find('-image')] + '-mask-r' + str(mask) + '.nii.gz'
                    new_filename = file
                    if mask==mask_count:
                        copy_and_rename(zip_path,file,train_image_dir,new_filename,delete_original = True)
                    else:
                        copy_and_rename(zip_path,file,train_image_dir,new_filename)
    #removing all other files installed due to the unzip
    elif file.endswith('.txt'):
        os.remove(os.path.join(task_folder_name,file))

In [33]:

train_files = os.listdir(train_image_dir)
label_files = os.listdir(train_label_dir)
print("train image files:",len(train_files))
print("train label files:",len(label_files))
print("Matches:",len(set(train_files).intersection(set(label_files))))

train image files: 215
train label files: 215
Matches: 0


In [22]:

#unzip the testing files in nnUNet_raw folder
os.chdir(task_folder_name)
!unzip testing.zip
os.chdir(base_dir)

Archive:  testing.zip
   creating: 2D scans test/
  inflating: __MACOSX/._2D scans test  
  inflating: 2D scans test/2011_13_0000.nii.gz  
  inflating: 2D scans test/2006_32_0000.nii.gz  
  inflating: 2D scans test/2006_27_0000.nii.gz  
  inflating: __MACOSX/2D scans test/._2006_27_0000.nii.gz  
  inflating: 2D scans test/2006_31_0000.nii.gz  
  inflating: 2D scans test/2006_28_0000.nii.gz  
  inflating: 2D scans test/2011_10_0000.nii.gz  
  inflating: 2D scans test/2011_12_0000.nii.gz  
  inflating: 2D scans test/2011_11_0000.nii.gz  
  inflating: 2D scans test/2006_29_0000.nii.gz  
  inflating: 2D scans test/2006_30_0000.nii.gz  


In [25]:
zip_path_test = os.path.join(task_folder_name,"2D scans test")
for file in os.listdir(zip_path_test):

    if file.endswith('0000.nii.gz'):
        #putting mask
        shutil.move(os.path.join(zip_path_test,file),test_dir)
    
    #removing all other files installed due to the unzip
    elif file.endswith('.txt'):
        os.remove(os.path.join(task_folder_name,file))
print("Testing files:",len(os.listdir(test_dir)))
print(test_dir)

Testing files: 10
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task101_IC/imagesTs


In [36]:
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:

    json_dict = OrderedDict()
    json_dict['name'] = task_name
    json_dict['description'] = "Internal Capsule segmentation"
    json_dict['tensorImageSize'] = "2D"
    json_dict['reference'] = "see challenge website"
    json_dict['licence'] = "see challenge website"
    json_dict['release'] = "0.0"

    #you may mention more than one modality
    json_dict['modality'] = {
        "0": "CT"
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "0": "background",
        "1": "white matter",
       
    }
    
    train_ids = os.listdir(train_label_dir)
    print(train_ids)
    test_ids = os.listdir(test_dir)
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTest'] = len(test_ids)

    #no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % (i[:i.find("_mask")]+'.nii.gz'), "label": "./labelsTr/%s" % i} for i in train_ids]
    #json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]
    #removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % (i[:i.find("_0000")]+'.nii.gz') for i in test_ids]

    with open(os.path.join(task_folder_name,"dataset.json"), 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json already exist!
['2357_14_mask.nii.gz', '2053_30_mask.nii.gz', '2200_18_mask.nii.gz', '2054_23_mask.nii.gz', '2345_20_mask.nii.gz', '2332_9_mask.nii.gz', '2053_29_mask.nii.gz', '2217_13_mask.nii.gz', '2035_15_mask.nii.gz', '2035_13_mask.nii.gz', '2175_16_mask.nii.gz', '2363_36_mask.nii.gz', '2338_14_mask.nii.gz', '2266_12_mask.nii.gz', '2134_34_mask.nii.gz', '2306_12_mask.nii.gz', '2086_14_mask.nii.gz', '2285_19_mask.nii.gz', '2110_25_mask.nii.gz', '2027_13_mask.nii.gz', '2257_29_mask.nii.gz', '2381_6_mask.nii.gz', '2394_15_mask.nii.gz', '2136_15_mask.nii.gz', '2217_11_mask.nii.gz', '2073_20_mask.nii.gz', '2363_34_mask.nii.gz', '2345_21_mask.nii.gz', '2146_19_mask.nii.gz', '2357_16_mask.nii.gz', '2394_16_mask.nii.gz', '2134_35_mask.nii.gz', '2200_19_mask.nii.gz', '2025_10_mask.nii.gz', '2198_17_mask.nii.gz', '2178_16_mask.nii.gz', '2285_18_mask.nii.gz', '2350_15_mask.nii.gz', '2027_14_mask.nii.gz', '2332_12_mask.nii.gz', '2198_16_mask.nii.gz', '2332_8_mask.nii.gz', '2144_18

In [37]:
os.chdir(main_dir)
!python experiment_planning/nnUNet_plan_and_preprocess.py -t 101 --verify_dataset_integrity
os.chdir(base_dir)



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Verifying training set
checking case 2357_14
Traceback (most recent call last):
  File "experiment_planning/nnUNet_plan_and_preprocess.py", line 170, in <module>
    main()
  File "experiment_planning/nnUNet_plan_and_preprocess.py", line 105, in main
    verify_dataset_integrity(join(nnUNet_raw_data, task_name))
  File "/home/olivia/Documents/tractTracker/nnUNet/nnunet/preprocessing/sanity_checks.py", line 130, in verify_dataset_integrity
    c, expected_label_file)
AssertionError: could not find label file for case 2357_14. Expected file: 
/home/olivia/Documents/tractTracker/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task